In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("FPA_FOD_Plus.csv", low_memory=False)
df.head()

,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,LOCAL_FIRE_REPORT_ID,...,erc_Percentile,NDVI-1day,NDVI_min,NDVI_max,NDVI_mean,CheatGrass,ExoticAnnualGrass,Medusahead,PoaSecunda,geometry
0,300300629,HIWMO-OA2805,INTERAGCY,IA-HIWMO,ST/C&L,USHIHNLX,City and County of Honolulu,HICNTY,Honolulu Fire Dept,NaN,...,NaN,0.27,'-0.03' '-0.01' '-0.07' '-0.09' '0.0' '-0.01' ...,'0.64' '0.7' '0.59' '0.71' '0.67' '0.74' '0.67...,'0.3' '0.28' '0.19' '0.27' '0.28' '0.27' '0.28...,NaN,NaN,NaN,NaN,NaN
1,687128,SFO-TX02240707-86179,NONFED,ST-NASF,ST/C&L,USTXTXS,Texas A & M Forest Service,TXTXS,Texas Forest Service,NaN,...,30-50%,0.35,'-0.1' '-0.03' '-0.05' '-0.02' '0.02' '0.0' '0...,'0.61' '0.61' '0.66' '0.59' '0.68' '0.48' '0.5...,'0.25' '0.32' '0.35' '0.31' '0.26' '0.28' '0.3...,NaN,NaN,NaN,NaN,NaN
2,319387,W-570451,FED,DOI-WFMI,BLM,USCACND,Central California District,CABBD,Bakersfield District,NaN,...,70-90%,NaN,'-0.06' '-0.04' '0.04' '0.12' '0.23' '0.33' '0...,'0.63' '0.61' '0.66' '0.64' '0.72' '0.68' '0.6...,'0.28' '0.25' '0.39' '0.47' '0.53' '0.53' '0.4...,NaN,NaN,NaN,NaN,NaN
3,686959,SFO-TX02240707-84214,NONFED,ST-NASF,ST/C&L,USTXTXS,Texas A & M Forest Service,TXTXS,Texas Forest Service,NaN,...,50-70%,0.28,'-0.09' '-0.01' '-0.03' '-0.01' '0.04' '0.02' ...,'0.61' '0.59' '0.65' '0.65' '0.65' '0.68' '0.4...,'0.13' '0.28' '0.26' '0.23' '0.23' '0.26' '0.2...,NaN,NaN,NaN,NaN,NaN
4,429152,SFO-2007FLFLS2007160001,NONFED,ST-NASF,ST/C&L,USFLFLS,Florida Forest Service,FLFLS,Florida Forest Service,NaN,...,<10%,0.08,'-0.08' '-0.06' '-0.06' '0.07' '-0.06' '-0.06'...,'0.53' '0.64' '0.64' '0.62' '0.56' '0.59' '0.5...,'0.18' '0.28' '0.31' '0.28' '0.22' '0.21' '0.2...,NaN,NaN,NaN,NaN,NaN


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2302521 entries, 0 to 2302520
Columns: 308 entries, FOD_ID to geometry
dtypes: float64(239), int64(10), object(59)
memory usage: 5.3+ GB


In [13]:
target = "NWCG_GENERAL_CAUSE"  
vars = [
    # NEW
    "FIRE_YEAR", "DISCOVERY_DOY",
    "FIPS_CODE", "Annual_etr", "Annual_precipitation", "Annual_tempreture", "pr", "tmmn", "vs",
    "fm100", "fm1000", "bi", "vpd", "erc", "erc_Percentile", "Elevation_1km", "Aspect_1km", "Slope_1km", "TPI_1km",
    "EVC_1km", "Evacuation", "SDI", "FRG", "FRG_1km", "No_FireStation_1.0km", "GACC_PL", "Mang_Name", "GAP_Sts", "GDP",
    "GHM", "NDVI-1day", "NPL", "Popo_1km", "road_county_dis", "RPL_THEMES", "RPL_THEME1", "RPL_THEME2", "RPL_THEME3", "RPL_THEME4"]

In [14]:
keep_cols = vars + [target]
df_model = df[[c for c in keep_cols if c in df.columns]].copy()
df_model.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2302521 entries, 0 to 2302520
Data columns (total 40 columns):
 #   Column                Dtype  
---  ------                -----  
 0   FIRE_YEAR             int64  
 1   DISCOVERY_DOY         int64  
 2   FIPS_CODE             float64
 3   Annual_etr            int64  
 4   Annual_precipitation  int64  
 5   Annual_tempreture     float64
 6   pr                    float64
 7   tmmn                  float64
 8   vs                    float64
 9   fm100                 float64
 10  fm1000                float64
 11  bi                    float64
 12  vpd                   float64
 13  erc                   float64
 14  erc_Percentile        object 
 15  Elevation_1km         float64
 16  Aspect_1km            float64
 17  Slope_1km             float64
 18  TPI_1km               float64
 19  EVC_1km               object 
 20  Evacuation            float64
 21  SDI                   float64
 22  FRG                   int64  
 23  FRG_1km

In [16]:
# Show all columns instead of truncated with ...
pd.set_option('display.max_columns', None)

# Optional: Show wider content or prevent string cutting
pd.set_option('display.max_colwidth', None)

# Optional: See more of long sequences (lists, etc.)
pd.set_option('display.max_seq_items', None)

# Now print your DataFrame head
print(df_model.head())

   FIRE_YEAR  DISCOVERY_DOY  FIPS_CODE  Annual_etr  Annual_precipitation  \
0       2007              1    15003.0           0                     0   
1       2007              1    48143.0        2009                  1175   
2       2007              1     6053.0        1625                   257   
3       2007              1    48013.0        2057                  1200   
4       2007              1    12093.0        2279                  1083   

   Annual_tempreture   pr        tmmn   vs  fm100  fm1000    bi   vpd   erc  \
0                NaN  NaN         NaN  NaN    NaN     NaN   NaN   NaN   NaN   
1              291.0  0.0  269.299988  3.2   17.7    17.9  28.0  0.35  28.0   
2              286.0  0.0  276.500000  6.8   14.7    19.4  38.0  0.47  27.0   
3              294.0  0.0  275.100006  2.7   16.0    17.4  29.0  0.61  33.0   
4              296.0  0.9  288.299988  4.6   19.1    20.5   0.0  0.85  12.0   

  erc_Percentile  Elevation_1km  Aspect_1km  Slope_1km  TPI_1km  \
0

In [20]:
import pandas as pd
import numpy as np

# Convert erc_Percentile values to numeric
def convert_erc_percentile(val):
    if pd.isna(val):
        return np.nan
    val = val.strip()
    if val.startswith('<'):
        try:
            num = float(val[1:].replace('%',''))
            return num / 2
        except:
            return np.nan
    elif '-' in val:
        parts = val.replace('%', '').split('-')
        try:
            lower, upper = float(parts[0]), float(parts[1])
            return (lower + upper) / 2
        except:
            return np.nan
    else:
        try:
            return float(val.replace('%',''))
        except:
            return np.nan

# Extract count and percentage from columns like EVC_1km, FRG_1km
def extract_count(val):
    try:
        return int(val.split('(')[0])
    except:
        return pd.NA

def extract_percentage(val):
    try:
        percent_str = val.split('(')[1].replace(')', '').replace('%', '')
        return float(percent_str)
    except:
        return pd.NA

# Apply transformations to df_model dataframe
def preprocess_dataframe(df_model):
   # Process erc_Percentile
    df_model['erc_Percentile_numeric'] = df_model['erc_Percentile'].apply(convert_erc_percentile)
    
    # Process EVC_1km columns
    df_model['EVC_1km_count'] = df_model['EVC_1km'].apply(extract_count).replace({pd.NA: np.nan})
    df_model['EVC_1km_percent'] = df_model['EVC_1km'].apply(extract_percentage).replace({pd.NA: np.nan})
    
    # Process FRG_1km columns
    df_model['FRG_1km_count'] = df_model['FRG_1km'].apply(extract_count).replace({pd.NA: np.nan})
    df_model['FRG_1km_percent'] = df_model['FRG_1km'].apply(extract_percentage).replace({pd.NA: np.nan})
    
    # Convert object columns that are potentially numeric strings
    obj_cols = df_model.select_dtypes(include='object').columns
    for col in obj_cols:
        if col in ['erc_Percentile', 'EVC_1km', 'FRG_1km']:
            # Already processed
            continue
        # Replace pd.NA with np.nan before conversion
        df_model[col] = df_model[col].replace({pd.NA: np.nan})
        try:
            df_model[col] = df_model[col].astype(float)
        except ValueError:
            # Leave as object if conversion fails
            pass

    return df_model

df_model = preprocess_dataframe(df_model)
df_model.head()

/tmp/ipykernel_57/1987862443.py:48: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_model['EVC_1km_count'] = df_model['EVC_1km'].apply(extract_count).replace({pd.NA: np.nan})
/tmp/ipykernel_57/1987862443.py:49: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_model['EVC_1km_percent'] = df_model['EVC_1km'].apply(extract_percentage).replace({pd.NA: np.nan})
/tmp/ipykernel_57/1987862443.py:52: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitl

,FIRE_YEAR,DISCOVERY_DOY,FIPS_CODE,Annual_etr,Annual_precipitation,Annual_tempreture,pr,tmmn,vs,fm100,fm1000,bi,vpd,erc,erc_Percentile,Elevation_1km,Aspect_1km,Slope_1km,TPI_1km,EVC_1km,Evacuation,SDI,FRG,FRG_1km,No_FireStation_1.0km,GACC_PL,Mang_Name,GAP_Sts,GDP,GHM,NDVI-1day,NPL,Popo_1km,road_county_dis,RPL_THEMES,RPL_THEME1,RPL_THEME2,RPL_THEME3,RPL_THEME4,NWCG_GENERAL_CAUSE,erc_Percentile_numeric,EVC_1km_count,EVC_1km_percent,FRG_1km_count,FRG_1km_percent
0,2007,1,15003.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,-9999,NaN,NaN,1.0,NaN,NaN,51481.170,0.53,0.27,1.0,5.7139,NaN,0.563,0.694,0.087,0.324,0.578,Missing data/not specified/undetermined,NaN,NaN,NaN,NaN,NaN
1,2007,1,48143.0,2009,1175,291.0,0.0,269.299988,3.2,17.7,17.9,28.0,0.35,28.0,30-50%,235.318881,62.606760,1.965501,0.00,136(3%) / 355(3%) / 25(2%),NaN,0.07,1,1(67%) / 2(33%),NaN,1.0,UNK,4.0,47810.750,0.21,0.35,1.0,0.0302,NaN,0.224,0.401,0.248,0.037,0.252,Debris and open burning,40.0,136.0,NaN,1.0,NaN
2,2007,1,6053.0,1625,257,286.0,0.0,276.500000,6.8,14.7,19.4,38.0,0.47,27.0,70-90%,85.913912,169.522268,11.784206,0.01,25(14%) / 11(13%) / 14(10%),NaN,0.00,1,1(46%) / 4(29%) / 111(12%),NaN,1.0,UNK,4.0,55688.996,0.42,NaN,1.0,1.1494,NaN,0.055,0.027,0.245,0.039,0.203,Misuse of fire by a minor,80.0,25.0,NaN,1.0,NaN
3,2007,1,48013.0,2057,1200,294.0,0.0,275.100006,2.7,16.0,17.4,29.0,0.61,33.0,50-70%,107.191716,-0.812544,0.567101,-0.00,355(14%) / 25(5%) / 17(4%),NaN,0.06,2,2(93%) / 1(7%),NaN,1.0,UNK,4.0,47810.750,0.49,0.28,1.0,0.4190,NaN,0.885,0.646,0.714,0.879,0.953,Equipment and vehicle use,60.0,355.0,NaN,2.0,NaN
4,2007,1,12093.0,2279,1083,296.0,0.9,288.299988,4.6,19.1,20.5,0.0,0.85,12.0,<10%,9.548252,-0.802797,-0.672261,-0.00,355(9%) / 25(5%) / 11(3%),NaN,0.12,1,2(40%) / 1(34%) / 5(24%),NaN,1.0,UNK,4.0,50591.914,0.31,0.08,1.0,0.1064,NaN,0.593,0.635,0.359,0.188,0.585,Fireworks,5.0,355.0,NaN,2.0,NaN


In [22]:
df_model = df_model.drop(columns=['erc_Percentile', 'EVC_1km', 'FRG_1km'])

In [23]:
df_model.to_csv('FPA-FOD_39attributes.csv')  